In [1]:
import time
import copy
import pickle

import numpy as np
import pandas as pd
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, r2_score, make_scorer

from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor

from xgboost import XGBRegressor

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [3]:
def rmse(estimator, X_eval, y_eval):
    y_hat = estimator.predict(X_eval)
    return np.sqrt(mean_squared_error(y_eval, y_hat))


def r2(estimator, X_eval, y_eval):
    y_hat = estimator.predict(X_eval)
    return r2_score(y_eval, y_hat)


def peason_r(estimator, X_eval, y_eval):
    y_hat = estimator.predict(X_eval)
    return np.corrcoef(y_eval, y_hat)[0, 1]


def peason_r_metric(y_true, y_pred):
    return np.corrcoef(y_true, y_pred)[0, 1]

peason_r_score = make_scorer(peason_r_metric)

In [4]:
def whole_to_int(a_dict):
    new_dict = copy.deepcopy(a_dict)
    for k, v in new_dict.items():
        if np.isclose(np.round(v), v):
            new_dict[k] = int(new_dict[k])
    return new_dict


def hyperopt_reg(regressor,
                 params_tuned, 
                 X_train, y_train,
                 num_eval,
                 params_fixed=None,
                 rstate=None):
    
    time_start = time.time()
    if params_fixed is None:
        params_fixed = {}
    
    def objective(params):
        regressor.set_params(**params_fixed, **params)
        # may use scoring='r2', "neg_mean_squared_error"
        neg_mse = cross_val_score(regressor, X_train, y_train, cv=10, scoring="neg_mean_squared_error").mean()
        #r2 = cross_val_score(regressor, X_train, y_train, cv=10, scoring="r2").mean()
        #pearson_r = cross_val_score(regressor, X_train, y_train, cv=10, scoring=peason_r_score).mean()
        return {"loss": -neg_mse, "status": STATUS_OK}
    
    if rstate is not None:
        rstate = np.random.RandomState(rstate)
        
    trials = Trials()
    best_params = fmin(objective, 
                      params_tuned, 
                      algo=tpe.suggest, 
                      max_evals=num_eval, 
                      trials=trials,
                      rstate=rstate)
    
    best_params = whole_to_int(best_params)
    best_model = regressor.set_params(**params_fixed, **best_params)
    best_model.fit(X_train, y_train)
    
    time_end = time.time()
    time_elapse = time_end - time_start
    print("Time elapsed: %0.5f s" % time_elapse)
    return trials, best_params, best_model

# Load train/test

In [6]:
pdXY = pd.read_csv("data/process/pdXY_rdkit_descriptors_123ft_clean.csv")
pdXY.head()

,MaxEStateIndex,MinEStateIndex,MinAbsEStateIndex,qed,MolWt,MaxPartialCharge,MinPartialCharge,MaxAbsPartialCharge,MinAbsPartialCharge,FpDensityMorgan1,...,fr_thiazole,fr_thiophene,fr_unbrch_alkane,fr_urea,id,smiles,dG,code,train_test,smiles_len
0,-0.692673,0.531768,-0.245712,0.064977,-1.951908,-1.129643,-0.627754,0.498481,-1.382592,-0.879653,...,-0.194933,-0.10042,-0.471332,-0.165383,0_,Oc1ccc(O)cc1,-12.213645,labeled_pubchem,train,12
1,-0.585287,0.539856,-0.216202,1.347428,-0.904952,-0.266570,0.365370,-0.369658,-0.276485,1.437874,...,5.121419,-0.10042,-0.471332,-0.165383,1_,Cc1c(CCO)sc[n+]1Cc1cnc(C)nc1N,-4.621139,labeled_pubchem,train,29
2,-0.036287,-3.851328,-0.744460,0.191786,-0.365599,1.654712,0.479265,0.197060,1.318963,1.471583,...,5.121419,-0.10042,-0.471332,-0.165383,2_,Cc1c(CCOP(O)(O)=O)sc[n+]1Cc1cnc(C)nc1N,-5.388212,labeled_pubchem,test,38
3,0.206395,-4.524025,0.019110,-0.525191,0.173753,1.745489,0.479265,0.286481,1.318963,0.981499,...,5.121419,-0.10042,-0.471332,-0.165383,3_,Cc1c(CCOP(O)(=O)OP(O)(O)=O)sc[n+]1Cc1cnc(C)nc1N,-9.614035,labeled_pubchem,train,47
4,-0.726608,0.585316,-0.050335,0.819691,-1.728699,-1.410163,0.365342,-0.369633,-1.742104,2.550287,...,5.121419,-0.10042,-0.471332,-0.165383,4_,Cc1ncsc1CCO,-3.479277,labeled_pubchem,train,11


In [7]:
PDY_COLS = ["id", "smiles", "dG", "code", "train_test", "smiles_len"]
PDX_COLS = [col for col in pdXY.columns if col not in PDY_COLS]
print(len(PDX_COLS))

X_train = pdXY.loc[pdXY["train_test"] == "train", PDX_COLS].copy().values
y_train = pdXY.loc[pdXY["train_test"] == "train", "dG"].copy().values
print(X_train.shape, y_train.shape)

X_test = pdXY.loc[pdXY["train_test"] == "test", PDX_COLS].copy().values
y_test = pdXY.loc[pdXY["train_test"] == "test", "dG"].copy().values
print(X_test.shape, y_test.shape)

123
(600, 123) (600,)
(162, 123) (162,)


In [8]:
pdXY.head()

,MaxEStateIndex,MinEStateIndex,MinAbsEStateIndex,qed,MolWt,MaxPartialCharge,MinPartialCharge,MaxAbsPartialCharge,MinAbsPartialCharge,FpDensityMorgan1,...,fr_thiazole,fr_thiophene,fr_unbrch_alkane,fr_urea,id,smiles,dG,code,train_test,smiles_len
0,-0.692673,0.531768,-0.245712,0.064977,-1.951908,-1.129643,-0.627754,0.498481,-1.382592,-0.879653,...,-0.194933,-0.10042,-0.471332,-0.165383,0_,Oc1ccc(O)cc1,-12.213645,labeled_pubchem,train,12
1,-0.585287,0.539856,-0.216202,1.347428,-0.904952,-0.266570,0.365370,-0.369658,-0.276485,1.437874,...,5.121419,-0.10042,-0.471332,-0.165383,1_,Cc1c(CCO)sc[n+]1Cc1cnc(C)nc1N,-4.621139,labeled_pubchem,train,29
2,-0.036287,-3.851328,-0.744460,0.191786,-0.365599,1.654712,0.479265,0.197060,1.318963,1.471583,...,5.121419,-0.10042,-0.471332,-0.165383,2_,Cc1c(CCOP(O)(O)=O)sc[n+]1Cc1cnc(C)nc1N,-5.388212,labeled_pubchem,test,38
3,0.206395,-4.524025,0.019110,-0.525191,0.173753,1.745489,0.479265,0.286481,1.318963,0.981499,...,5.121419,-0.10042,-0.471332,-0.165383,3_,Cc1c(CCOP(O)(=O)OP(O)(O)=O)sc[n+]1Cc1cnc(C)nc1N,-9.614035,labeled_pubchem,train,47
4,-0.726608,0.585316,-0.050335,0.819691,-1.728699,-1.410163,0.365342,-0.369633,-1.742104,2.550287,...,5.121419,-0.10042,-0.471332,-0.165383,4_,Cc1ncsc1CCO,-3.479277,labeled_pubchem,train,11


In [ ]:
X_all = pdXY[PDX_COLS].copy().values

# Linear regression

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)

print("Train RMSE:", rmse(lr, X_train, y_train))
print("Train Pearson's R:", peason_r(lr, X_train, y_train))

print("Test RMSE:", rmse(lr, X_test, y_test))
print("Test Pearson's R:", peason_r(lr, X_test, y_test))

In [ ]:
ridge = Ridge()

params = {"alpha": hp.loguniform("alpha", np.log(1e-10), np.log(1e10)),}

num_eval = 100

trials, best_params, best_model = hyperopt_reg(ridge, params, X_train, y_train, num_eval)
print("best_params:", best_params)

print("Train RMSE:", rmse(best_model, X_train, y_train))
print("Train Pearson's R:", peason_r(best_model, X_train, y_train))

print("Test RMSE:", rmse(best_model, X_test, y_test))
print("Test Pearson's R:", peason_r(best_model, X_test, y_test))

pickle.dump(best_model, open("models/lr_01.pkl", "wb"))

In [ ]:
model = pickle.load(open("models/lr_01.pkl", "rb"))
y_test_pred = model.predict(X_test)
test_pred_df = pd.DataFrame({"dG": y_test, "pred": y_test_pred})
test_pred_df.to_csv("results/lr/test_pred.csv", index=False)

# Random Forest

In [ ]:
params = {
    "max_depth": scope.int(hp.quniform("max_depth", 2, 14, 1)),
    "min_samples_split": scope.int(hp.quniform("min_samples_split", 2, 20, 2)),
    "min_samples_leaf": scope.int(hp.quniform("min_samples_leaf", 2, 20, 2)), 
    "max_features": scope.int(hp.quniform("max_features", 10, 60, 5)),
}

params_fixed = {
    "n_estimators": 1000
}


num_eval = 100

rf = RandomForestRegressor()

trials, best_params, best_model = hyperopt_reg(rf, params, X_train, y_train, num_eval, params_fixed=params_fixed)
print("best_params:", best_params)

print("Train RMSE:", rmse(best_model, X_train, y_train))
print("Train Pearson's R:", peason_r(best_model, X_train, y_train))

print("Test RMSE:", rmse(best_model, X_test, y_test))
print("Test Pearson's R:", peason_r(best_model, X_test, y_test))

pickle.dump(best_model, open("models/rf_01.pkl", "wb"))

In [ ]:
model = pickle.load(open("models/rf_01.pkl", "rb"))
y_test_pred = model.predict(X_test)
test_pred_df = pd.DataFrame({"dG": y_test, "pred": y_test_pred})
test_pred_df.to_csv("results/rf/test_pred.csv", index=False)

# XGBOOST

In [ ]:
params = {
    "max_depth": scope.int(hp.quniform("max_depth", 2, 8, 1)),
    "min_child_weight": scope.int(hp.quniform("min_child_weight", 1, 10, 1)), 
    "subsample": hp.uniform("subsample", 0.4, 1.0),
    "colsample_bytree": hp.uniform("colsample_bytree", 0.4, 1.0),
    "reg_lambda": hp.loguniform("reg_lambda", np.log(0.00001), np.log(100)),
    #"reg_alpha": hp.loguniform("reg_alpha", np.log(0.001), np.log(1000)),
    "learning_rate": hp.loguniform("learning_rate", np.log(0.0001), np.log(1.)),
    #"gamma": hp.uniform("gamma", 0., 5.),
}

params_fixed = {
    "tree_method": "gpu_hist",
    "predictor": "gpu_predictor",
    "n_estimators": 300
}


num_eval = 100

xgb = XGBRegressor()

trials, best_params, best_model = hyperopt_reg(xgb, params, X_train, y_train, num_eval, params_fixed=params_fixed)
print("best_params:", best_params)

print("Train RMSE:", rmse(best_model, X_train, y_train))
print("Train Pearson's R:", peason_r(best_model, X_train, y_train))

print("Test RMSE:", rmse(best_model, X_test, y_test))
print("Test Pearson's R:", peason_r(best_model, X_test, y_test))

pickle.dump(best_model, open("models/xbg_01.pkl", "wb"))

In [ ]:
model = pickle.load(open("models/xbg_01.pkl", "rb"))
y_test_pred = model.predict(X_test)
test_pred_df = pd.DataFrame({"dG": y_test, "pred": y_test_pred})
test_pred_df.to_csv("results/xgb/test_pred.csv", index=False)